# Memuat data dari Piksel

* **Product yang digunakan:** 
[s2_l2a](http://43.218.254.133:8080/products/s2_l2a), 
[ls9_c2l2_sr](http://43.218.254.133:8080/products/ls9_c2l2_sr)

* **Prasyarat:** Pengguna notebook ini harus sudah memiliki pemahaman dasar tentang:
 * Cara menjalankan [Jupyter notebook](01_Jupyter_notebooks.ipynb)
 * Memeriksa ketersediaan [Product dan Measurements Piksel](02_Products_and_measurements.ipynb)

**Kata Kunci** Indeks: `panduan pemula; memuat data`, :indeks:`memuat data; panduan pemula`, :indeks:`data yang digunakan; landsat 9`, :indeks:`data yang digunakan; Sentinel 2`, :indeks:`metode data; resampling`, :indeks:`metode data; reproyeksi`, :indeks:`atribut data; sistem referensi koordinat`

## Latar belakang
Memuat data dari instans [Piksel](http://43.218.254.133:8080/) pada [Open Data Cube](https://www.opendatacube.org/) memerlukan pembuatan query data yang menentukan apa (what), di mana (where), dan kapan (when) data tersebut diminta. Setiap query akan mengembalikan [objek xarray multi-dimensi](http://xarray.pydata.org/en/stable/) yang berisi hasil dari query tersebut. 

Pemahaman tentang struktur data `xarray` sangat penting karena merupakan dasar dari struktur data yang dimuat dari datacube.  Manipulasi, transformasi, dan visualisasi objek `xarray` memberikan pengguna datacube kemampuan untuk menjelajahi dan menganalisis dataset Piksel, serta mengajukan dan menjawab pertanyaan-pertanyaan ilmiah.

## Deskripsi
Notebook ini akan memperkenalkan cara memuat data dari datacube Piksel melalui pembuatan query dan penggunaan fungsi `dc.load()`.
Topik yang dibahas meliputi:

* Memuat data menggunakan `dc.load()`
* Penjelasan `xarray.Dataset`
    * Memeriksa sebuah `xarray.DataArray` individu
* Kustomisasi fungsi `dc.load()`
    * Menentukan measurements
    * Metode resampling spasial
* Memuat data menggunakan dictionary
* Penggunaan `like`
* Menambahkan progress bar 

***

## Memulai
Untuk menjalankan pengenalan tentang memuat data dari Piksel ini, jalankan semua sel di notebook mulai dari sel "Memuat paket". Untuk bantuan dalam menjalankan sel notebook, lihat kembali [notebook Jupyter Notebooks](01_Jupyter_notebooks.ipynb).

### Memuat Paket
Pertama, kita perlu memuat paket `datacube`. Ini akan memungkinkan kita untuk mengakses basisdata datacube dan memuat beberapa data. Fungsi `with_ui_cbk` dari `odc.ui` memungkinkan kita untuk menampilkan progress bar saat memuat data dalam jumlah besar.

In [37]:
import datacube
from odc.ui import with_ui_cbk

### Terhubung ke datacube
Gunakan objek datacube `dc` untuk memuat data.
Parameter `app` adalah nama unik yang digunakan untuk mengidentifikasi notebook yang tidak memiliki efek apa pun pada analisis.

In [38]:
dc = datacube.Datacube(app="03_Loading_data")

### Definisi variabel 

* `lat_range`: jangkauan/rentang wilayah spasial garis lintang (latitude) yang merupakan  garis imajiner yang melintang horizontal di permukaan Bumi. 
* `lon_range`: jangkauan/rentang wilayah spasial garis bujur (longitude) yang merupakan garis imajiner yang membentang dari kutub utara ke kutub selatan Bumi.
   Dimensi ``longitude``/``latitude`` dan ``x``/``y`` dapat digunakan secara bergantian.
* `time_range`: rentang waktu. Dimensi waktu dapat ditentukan menggunakan tuple dari objek datetime atau string dalam format "YYYY", "YYYY-MM" atau "YYYY-MM-DD".

In [39]:
lat_range = (-6.000237, -6.020759)
lon_range = (106.596046, 106.632157)
time_range = ("2024-01-01", "2024-10-31")

## Memuat data menggunakan `dc.load()`

Memuat data dari datacube menggunakan fungsi [dc.load()](https://datacube-core.readthedocs.io/en/latest/api/indexed-data/generate/datacube.Datacube.load.html). Fungsi ini memerlukan argumen minimum berikut:

* `product`: Produk tertentu yang akan dimuat (untuk meninjau produk Piksel, lihat notebook [Produk dan Band](02_Products_and_measurements.ipynb)).
* `x`: Mengambil nilai dari variabel `lat_range` yang menerima query dalam sistem koordinat geografis WGS84.
* `y`: Mengambil nilai dari variabel `lon_range`.
* `time`: Mengambil nilai dari variabel `time_range`.

Argumen opsional yang memberikan kemudahan penggunaan dan identifikasi band yang akan dimuat adalah:

* `measurements:` Argumen ini digunakan untuk memberikan daftar nama band yang akan dimuat, seperti yang tercantum dalam `dc.list_measurements()`. 
Untuk dataset satelit, `measurements` berisi data untuk setiap band satelit individu (misalnya: red, green, blue, near infrared). 
Jika tidak disediakan, semua band untuk produk akan dikembalikan, dan akan memiliki nama default dari data satelit tersebut.

Mari jalankan query untuk memuat data tahun 2024 dari [produk Sentinel 2 s2_l2a](http://43.218.254.133:8080/products/s2_l2a) untuk daerah sebagian Tangerang. 
Untuk contoh ini, kita dapat menggunakan parameter berikut:

* `product`: `s2_l2a`
* `x`=`lon_range`
* `y`=`lat_range` 
* `time`=`time_range`
* `measurements`= `['red', 'green', 'blue', 'nir']` 

Jalankan sel berikut untuk memuat semua dataset dari produk `s2_l2a` yang sesuai dengan rentang spasial dan temporal ini:

In [40]:
ds = dc.load(
    product='s2_l2a',  # Ganti dengan nama produk sesuai konfigurasi ODC Anda
    x=lon_range,  # Longitude
    y=lat_range,  # Latitude
    time=time_range,  # Rentang waktu
    measurements=['red', 'green', 'blue', 'nir'],  # Band
    output_crs='EPSG:32748',  # Proyeksi UTM untuk zona 48S
    resolution=(-10, 10),  # Resolusi spasial 10 meter
)
print(ds)

<xarray.Dataset> Size: 46MB
Dimensions:      (time: 62, y: 229, x: 402)
Coordinates:
  * time         (time) datetime64[ns] 496B 2024-01-04T03:19:40.454000 ... 20...
  * y            (y) float64 2kB 9.337e+06 9.337e+06 ... 9.334e+06 9.334e+06
  * x            (x) float64 3kB 6.766e+05 6.767e+05 ... 6.806e+05 6.807e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 11MB 7912 7880 7896 7880 ... 636 625 650
    green        (time, y, x) uint16 11MB 7872 7848 7832 7848 ... 787 786 794
    blue         (time, y, x) uint16 11MB 7896 7904 7896 7856 ... 844 813 833
    nir          (time, y, x) uint16 11MB 8280 8264 8280 8240 ... 1194 1084 1084
Attributes:
    crs:           epsg:32748
    grid_mapping:  spatial_ref


### Penjelasan `xarray.Dataset`
Variabel `ds` yang telah dijalankan menghasilkan sebuah `xarray.Dataset` yang berisi semua data yang sesuai dengan parameter query spasial dan temporal yang dimasukkan ke dalam `dc.load`.

*Dimensi*

* Mengidentifikasi jumlah waktu yang dikembalikan dalam pencarian (`time: 1`) serta jumlah piksel dalam `x` dan `y` dari query data.

*Koordinat*

* `time` mengidentifikasi atribut tanggal yang dikaitkan dengan setiap waktu yang dikembalikan.
* `x` dan `y` adalah koordinat untuk setiap piksel dalam batas query spasial.

*Variabel Data*

* Ini adalah band yang tersedia untuk produk yang telah ditentukan.
Untuk setiap tanggal (`time`) yang dikembalikan oleh query, nilai yang diukur pada setiap piksel (`y`, `x`) dikembalikan sebagai array untuk setiap band.
Setiap variabel data itu sendiri adalah objek `xarray.DataArray` ([lihat di bawah](#Memeriksa-individual-xarray.DataArray)).

*Atribut*

* `crs` mengidentifikasi sistem referensi koordinat (CRS) dari data yang dimuat.

### Memeriksa sebuah `xarray.DataArray` individu
`xarray.Dataset` yang dimuat di atas adalah kumpulan dari objek `xarray.DataArray` individu yang menyimpan data aktual untuk setiap variabel data/band. 

Sebagai contoh, semua band yang tercantum di bawah _Data variables_ di atas (misalnya `blue`, `green`, `red`, `nir`, `swir_1`, `swir_2`) adalah objek `xarray.DataArray`.

Kita dapat memeriksa data dalam objek `xarray.DataArray` ini menggunakan salah satu sintaks berikut:
```
ds["nama_band"]
```
atau:
```
ds.nama_band
```

Kemampuan untuk mengakses data dari band individu memungkinkan kita untuk memanipulasi dan menganalisis data dari band satelit individu atau lapisan tertentu dalam sebuah dataset. Sebagai contoh, kita dapat mengakses data dari band satelit infra-merah dekat (yaitu `nir`):

In [41]:
print(ds.nir)

<xarray.DataArray 'nir' (time: 62, y: 229, x: 402)> Size: 11MB
array([[[8280, 8264, 8280, ..., 7320, 7304, 7296],
        [8280, 8280, 8248, ..., 7304, 7304, 7296],
        [8248, 8240, 8248, ..., 7328, 7320, 7304],
        ...,
        [8448, 8368, 8384, ..., 6928, 6932, 6900],
        [8432, 8392, 8400, ..., 6924, 6928, 6904],
        [8352, 8368, 8384, ..., 6908, 6920, 6932]],

       [[8280, 8256, 8280, ..., 7320, 7296, 7296],
        [8280, 8280, 8248, ..., 7304, 7304, 7296],
        [8248, 8240, 8248, ..., 7328, 7328, 7312],
        ...,
        [8448, 8368, 8384, ..., 6928, 6928, 6888],
        [8424, 8400, 8400, ..., 6924, 6928, 6904],
        [8352, 8368, 8384, ..., 6908, 6908, 6932]],

       [[3776, 3768, 3744, ..., 3896, 3912, 3900],
        [3796, 3796, 3776, ..., 3852, 3872, 3896],
        [3764, 3776, 3784, ..., 3864, 3920, 3852],
        ...,
...
        ...,
        [ 542,  536,  554, ..., 2524, 1926, 1224],
        [ 556,  563,  554, ..., 4052, 3108, 1882],
        [ 

Perhatikan bahwa header objek memberi tahu kita bahwa itu adalah `xarray.DataArray` yang berisi data untuk measurements satelit `nir`.

Seperti `xarray.Dataset`, array ini juga mencakup informasi tentang **dimensi** data (yaitu `(time: 1, y: 801, x: 644)`), **koordinat**, dan **atribut**.
Variabel data khusus ini mengandung beberapa informasi tambahan yang spesifik untuk band `nir`, termasuk detail nilai nodata array (yaitu `nodata: -999`).

> **Catatan**: Untuk pengenalan yang lebih mendalam tentang struktur data `xarray`, lihat [dokumentasi resmi xarray](http://xarray.pydata.org/en/stable/data-structures.html).

## Kustomisasi fungsi `dc.load()`

Fungsi `dc.load()` dapat disesuaikan untuk memperbaiki query. Opsi penyesuaian meliputi:

* `measurements:` Argumen ini digunakan untuk memberikan daftar nama measurements yang akan dimuat, seperti yang tercantum dalam `dc.list_measurements()`. 
Untuk dataset satelit, measurements berisi data untuk setiap band satelit individu (misalnya, near infrared). 
Jika tidak disediakan, semua measurements untuk produk akan dikembalikan.
* `crs:` Sistem referensi koordinat (CRS) dari query `x` dan `y` memiliki nilai default `WGS84` kecuali bidang parameter `output_crs` ditentukan secara manual. 
* `group_by:` Dataset satelit yang berbasis pada tampilan dapat memiliki beberapa observasi per hari dengan rentang waktu yang sedikit berbeda saat satelit mengumpulkan data sepanjang jalurnya. Observasi ini dapat digabungkan dengan mengurangi dimensi `time` ke tingkat harian menggunakan `group_by=solar_day`.
* `output_crs` dan `resolution`: Untuk memproyeksikan ulang atau mengubah resolusi data, berikan bidang `output_crs` dan `resolution`.    
* `resampling`: Argumen ini memungkinkan Anda untuk menentukan metode resampling spasial khusus yang akan digunakan saat data diproyeksikan ke CRS yang berbeda.

Contoh sintaks tentang penggunaan opsi-opsi ini dapat dilihat di sel-sel di bawah ini.

> Untuk bantuan atau opsi penyesuaian lebih lanjut, jalankan `help(dc.load)` di sel kosong atau kunjungi [halaman dokumentasi](https://datacube-core.readthedocs.io/en/latest/api/indexed-data/generate/datacube.Datacube.load.html) fungsi tersebut.

### Menentukan measurements
Secara default, `dc.load()` akan memuat *semua* measurements dalam suatu produk. Untuk memuat data hanya dari band satelit `red`, `green`, dan `blue`, kita dapat menambahkan `measurements=["red", "green", "blue"]` ke dalam query kita:

In [42]:
ds_rgb = dc.load(product='s2_l2a',
                 measurements=["red", "green", "blue"],
                 x=lon_range,  # Longitude
                 y=lat_range,  # Latitude
                 time=time_range,  # Rentang waktu
                 output_crs='EPSG:32748',  # Proyeksi UTM untuk zona 48S
                 resolution=(-10, 10),  # Resolusi spasial 10 meter
)
print(ds_rgb)

<xarray.Dataset> Size: 34MB
Dimensions:      (time: 62, y: 229, x: 402)
Coordinates:
  * time         (time) datetime64[ns] 496B 2024-01-04T03:19:40.454000 ... 20...
  * y            (y) float64 2kB 9.337e+06 9.337e+06 ... 9.334e+06 9.334e+06
  * x            (x) float64 3kB 6.766e+05 6.767e+05 ... 6.806e+05 6.807e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 11MB 7912 7880 7896 7880 ... 636 625 650
    green        (time, y, x) uint16 11MB 7872 7848 7832 7848 ... 787 786 794
    blue         (time, y, x) uint16 11MB 7896 7904 7896 7856 ... 844 813 833
Attributes:
    crs:           epsg:32748
    grid_mapping:  spatial_ref


Perhatikan bahwa komponen *Data Variabel* dari `xarray.Dataset` sekarang hanya mencakup band yang ditentukan dalam query (yaitu band `red`, `green`, dan `blue`).

In [ ]:
ds_custom_crs = dc.load(product='s2_l2a',
                        x=lon_range,  # Longitude
                        y=lat_range,  # Latitude
                        time=time_range,  # Rentang waktu
                        crs='EPSG:32748'
                        
)
print(ds_custom_crs)

In [ ]:
ds_reprojected = dc.load(product='s2_l2a',  # Ganti dengan nama produk sesuai konfigurasi ODC Anda
    x=lon_range,  # Longitude
    y=lat_range,  # Latitude
    time=time_range,  # Rentang waktu
    output_crs='EPSG:32734',  # Proyeksi UTM untuk zona 48S
    resolution=(-10, 10),  # Resolusi spasial 10 meter
)

print(ds_reprojected)

Perhatikan bahwa atribut `crs` di bagian *Atribut* telah berubah menjadi `EPSG:32734`.  
Karena resolusi spasial juga telah didefinisikan, jumlah piksel pada dimensi `x` dan `y` juga berubah.

### Metode resampling spasial
Ketika suatu produk diproyeksikan ulang ke CRS dan/atau resolusi yang berbeda, grid piksel baru mungkin berbeda dari piksel input asli (dalam hal ukuran, jumlah, dan penjajaran).
Oleh karena itu, perlu diterapkan aturan "resampling" spasial yang mengalokasikan nilai piksel input ke dalam grid piksel baru.

Secara default, `dc.load()` melakukan resampling nilai piksel menggunakan metode "nearest neighbour", yang mengalokasikan setiap piksel baru dengan nilai dari piksel input terdekat.
Tergantung pada jenis data dan analisis yang dilakukan, ini mungkin bukan pilihan yang paling tepat (misalnya, untuk data kontinu).

Parameter `resampling` dalam `dc.load()` memungkinkan Anda untuk memilih metode resampling kustom dari opsi berikut:

```
"nearest", "cubic", "bilinear", "cubic_spline", "lanczos", 
"average", "mode", "gauss", "max", "min", "med", "q1", "q3"
```

Sebagai contoh, kita dapat meminta agar semua data yang dimuat di-resampling menggunakan metode "average":

In [43]:
# Note the additional `resampling` parameter
ds_averageresampling = dc.load(product="s2_l2a",
             x=(106.83936, 106.85936), 
             y=(-6.48081, -6.50081),
             time=("2024-07-01", "2024-07-10"),
             output_crs='EPSG:32748',
             resolution=(-10, 10),
             measurements=["red", "green", "blue"],
             resampling="average")
print(ds_averageresampling)

<xarray.Dataset> Size: 1MB
Dimensions:      (time: 4, y: 223, x: 223)
Coordinates:
  * time         (time) datetime64[ns] 32B 2024-07-02T03:19:58.263000 ... 202...
  * y            (y) float64 2kB 9.283e+06 9.283e+06 ... 9.281e+06 9.281e+06
  * x            (x) float64 2kB 7.034e+05 7.034e+05 ... 7.056e+05 7.056e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 398kB 3476 3472 3444 ... 2498 2480 2510
    green        (time, y, x) uint16 398kB 3836 3820 3836 ... 2652 2622 2624
    blue         (time, y, x) uint16 398kB 4292 4340 4312 ... 2742 2678 2646
Attributes:
    crs:           epsg:32748
    grid_mapping:  spatial_ref


Anda juga dapat menyediakan library Python untuk meminta metode pengambilan sampel yang berbeda untuk pengukuran yang berbeda. 
Ini bisa sangat berguna ketika beberapa band mengandung kategori data yang memerlukan metode pengambilan sampel ulang seperti "nearest" (terdekat) atau "mode" (modus) yang tidak mengubah nilai piksel input.

Kita juga dapat menentukan `resampling={"red": "nearest", "*": "average"}`, yang akan menggunakan pengambilan sampel ulang "nearest" (tetangga terdekat) hanya untuk band satelit `red`. `"*": "average"` akan menerapkan pengambilan sampel ulang "average" (rata-rata) untuk semua band satelit lainnya:

> **Catatan**: Untuk informasi lebih lanjut tentang metode pensampelan ulang spasial, lihat [panduan berikut](https://rasterio.readthedocs.io/en/stable/topics/resampling.html)

## Memuat data menggunakan dictionary
Seringkali berguna untuk menggunakan kembali seperangkat parameter query untuk memuat data dari beberapa produk.
Untuk mencapai ini, kita dapat memuat data menggunakan format dictionary.
Ini melibatkan penempatan parameter query yang kita gunakan untuk memuat data di atas ke dalam objek kamus Python yang dapat kita gunakan kembali untuk beberapa pemuatan data:

In [44]:
query = {"x": (106.83936, 106.85936),
         "y": (-6.48081, -6.50081),
         "time": ("2024-07-01", "2024-07-10"),
         "output_crs": 'EPSG:32748',
         "resolution": (-10, 10),
         "measurements": ["red", "green", "blue"]}

Kita kemudian dapat menggunakan dictionary sebagai input untuk `dc.load()`. Untuk informasi lebih lanjut tentang operator pembongkaran, lihat [dokumentasi Python](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists).

In [45]:
ds = dc.load(product="s2_l2a",
             **query)

print(ds)

<xarray.Dataset> Size: 1MB
Dimensions:      (time: 4, y: 223, x: 223)
Coordinates:
  * time         (time) datetime64[ns] 32B 2024-07-02T03:19:58.263000 ... 202...
  * y            (y) float64 2kB 9.283e+06 9.283e+06 ... 9.281e+06 9.281e+06
  * x            (x) float64 2kB 7.034e+05 7.034e+05 ... 7.056e+05 7.056e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 398kB 3476 3472 3444 ... 2498 2480 2510
    green        (time, y, x) uint16 398kB 3836 3820 3836 ... 2652 2622 2624
    blue         (time, y, x) uint16 398kB 4292 4340 4312 ... 2742 2678 2646
Attributes:
    crs:           epsg:32748
    grid_mapping:  spatial_ref


Sekarang kita memiliki kueri yang dapat digunakan kembali, kita dapat dengan mudah menggunakannya untuk memuat data dari produk yang berbeda.  
Sebagai contoh, kita dapat memuat data Ringkasan Tahunan Water Observations from Space (WOfS) untuk cakupan, waktu, CRS output, dan resolusi yang sama.

## Trik bermanfaat lainnya
### Penggunaan like
Opsi lain untuk memuat data yang sesuai dari beberapa produk adalah dengan menggunakan parameter `like` pada `dc.load()`.
Ini akan menyalin cakupan spasial dan temporal serta CRS/resolusi dari dataset yang sudah ada, dan menggunakan parameter ini untuk memuat data baru dari produk baru.

Dalam contoh di bawah ini, kita memuat dataset WOfS lain yang persis sesuai dengan dataset `ds_s2` yang telah kita muat sebelumnya:

```python
ds_landsat = dc.load(product="ls9_c212_sr",
                  like=ds_s2)
```

### Menambahkan progress bar
Saat memuat data dalam jumlah besar, dapat berguna untuk melihat kemajuan proses pemuatan data. 
Parameter `progress_cbk` dalam `dc.load()` memungkinkan kita untuk menambahkan progress bar yang akan menunjukkan sejauh mana proses pemuatan berlangsung.

In [47]:
query = {"x": (106.83936, 106.85936),
         "y": (-6.48081, -6.50081),
         "time": ("2024-07-01", "2024-07-10"),
         "output_crs": 'EPSG:32748',
         "resolution": (-10, 10),
         "measurements": ["red", "green", "blue"]}

ds_progress = dc.load(product="s2_l2a",
             progress_cbk=with_ui_cbk(),
             **query)

print(ds_progress)

<xarray.Dataset> Size: 1MB
Dimensions:      (time: 4, y: 223, x: 223)
Coordinates:
  * time         (time) datetime64[ns] 32B 2024-07-02T03:19:58.263000 ... 202...
  * y            (y) float64 2kB 9.283e+06 9.283e+06 ... 9.281e+06 9.281e+06
  * x            (x) float64 2kB 7.034e+05 7.034e+05 ... 7.056e+05 7.056e+05
    spatial_ref  int32 4B 32748
Data variables:
    red          (time, y, x) uint16 398kB 3476 3472 3444 ... 2498 2480 2510
    green        (time, y, x) uint16 398kB 3836 3820 3836 ... 2652 2622 2624
    blue         (time, y, x) uint16 398kB 4292 4340 4312 ... 2742 2678 2646
Attributes:
    crs:           epsg:32748
    grid_mapping:  spatial_ref


## Langkah selanjutnya 

Untuk informasi lebih lanjut yang lebih advanced tentang bekerja dengan Jupyter Notebooks atau JupyterLab, Anda dapat menjelajahi [halaman dokumentasi JupyterLab](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html).

Untuk melanjutkan bekerja melalui notebook dalam panduan pemula ini, notebook berikut dirancang untuk dikerjakan dalam urutan berikut:

1. [Jupyter Notebooks](01_Jupyter_notebooks.ipynb)
2. [Product dan Measurements](02_Product_dan_measurement.ipynb)
3. **Memuat data (notebook ini)**
4. [Plotting](04_Plotting.ipynb)
5. [Melakukan analisis dasar](05_Basic_analysis.ipynb)
6. [Pengenalan numpy](06_Intro_to_numpy.ipynb)
7. [Pengenalan xarray](07_Intro_to_xarray.ipynb)
8. [Pemrosesan paralel dengan Dask](08_Parallel_processing_with_dask.ipynb)



***

## Informasi Tambahan

**Lisensi:** Kode dalam notebook ini dilisensikan di bawah [Lisensi Apache, Versi 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Kontak:** Jika Anda memerlukan bantuan, silakan ajukan pertanyaan di [Saluran Slack Open Data Cube](http://slack.opendatacube.org/) atau di [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) menggunakan tag `open-data-cube` (Anda dapat melihat pertanyaan yang sebelumnya diajukan [di sini](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
Jika Anda ingin melaporkan masalah dengan notebook ini, Anda dapat mengajukannya di [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Versi datacube yang kompatibel:**

In [48]:
print(datacube.__version__)

1.8.19


**Terakhir dijalankan:**

In [49]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2025-02-10'